<a href="https://colab.research.google.com/github/fmigas/Projects/blob/main/names_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The objective of this project is to check how adding randomnes to a text-generating model adds quality to the end product.

The basis of the model is a simple LSTM network. Its task is to learn English and Polish names and later generate names using different schemes.

Let's start with the imports.

In [21]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.utils import np_utils
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import os
os. environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

Let's start with loading data - Polish and English names.

In [22]:
pnames = pd.read_csv('/content/drive/MyDrive/Python_data/P_names.csv')
enames = pd.read_csv('/content/drive/MyDrive/Python_data/E_names.csv')

In [23]:
print(f'Number of Polish names in a database: {len(pnames)}')
print(f'Number of English names in a database: {len(enames)}')

Number of Polish names in a database: 1711
Number of English names in a database: 18238


We have over 10 times as many English names as Polish names. We'll see how it affects the quality of what our model can produce.

A fan fact for English readers. In the USA if you have a kid, you can use any word that you wish as his/her name, your fantasty and your warm feelings towards your kid's future in a school is your limit.

In Poland (and probably in many other countries) you can only choose from an officially approved list of authorized names. Probably that is a reason why a list of Polish names is over 10 times shorter.

In [40]:
enames.head()

,name,len
0,michael,7
1,christopher,11
2,jessica,7
3,matthew,7
4,ashley,6


I preprocessed both files beforehead to save your time here. In both files we have no duplicates, all names lower cased and a 'len' column added with a char count for each name.

Our model is naturally a char-level generating model. It's core architecture and data stracture can be easily copied from any deep learning manual.

What is interesting here is how we apply randomness at two levels:
1. at the learning process by replacing some "next char" labels with random chars
2. at the generating process by applying what I call a "simplified Beam" in place of a greedy search routine

Will these two result in better, richer, more interesting and natural names? Let's see!

Here we defind a couple of constans. Three of them define randomness level.

In [25]:
SERIES = 10
EPOCHS = 20
RAND_Y = 10 # it defines every nth char (here every 10th) in y (labels) column will be replaced with a random char
RANDOM_CHARS = False # if False, we'll follow a greedy search path; if True, we'll add some "simplified beam" effect
MULTIPLIER = 3 # mocne podbicie znaczenia znaków o większym prawdopodobieństwie w predict

Let's start with English names.

In [26]:
names = enames

We need a full list of chars used in English names and length of the longest name.
As a standard procedure in such models, we also build char_to_int and int_to_char dictionaries.

In [27]:
max_len = names['len'].max()

nlist = list(names['name'])
text = ' '.join(nlist)
chars = sorted(list(set(text)))
print(f'The longest name is {max_len} chars long.')
print(f"Chars list: {chars}")
print(f"Number of unique chars, including white space: {len(chars)}")

char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

The longest name is 15 chars long.
Chars list: [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Number of unique chars, including white space: 27


Now we begin to build our X and Y datasets.

In [28]:
dataX = [] # x data
dataY_original = [] # y data (labels) - simply the next char

for name in names['name']:
    for i in range(len(name)):
        if i < len(name) - 1:
            dataX.append([char_to_int[char] for char in name[0:i+1]])
            dataY_original.append(char_to_int[name[i+1]])
        else:
            dataX.append([char_to_int[char] for char in name[0:i+1]])
            dataY_original.append(0)

print('X data')
print(dataX[:20])
print('Y data')
print(dataY_original[:20])
originalY = dataY_original[:20]

X data
[[13], [13, 9], [13, 9, 3], [13, 9, 3, 8], [13, 9, 3, 8, 1], [13, 9, 3, 8, 1, 5], [13, 9, 3, 8, 1, 5, 12], [3], [3, 8], [3, 8, 18], [3, 8, 18, 9], [3, 8, 18, 9, 19], [3, 8, 18, 9, 19, 20], [3, 8, 18, 9, 19, 20, 15], [3, 8, 18, 9, 19, 20, 15, 16], [3, 8, 18, 9, 19, 20, 15, 16, 8], [3, 8, 18, 9, 19, 20, 15, 16, 8, 5], [3, 8, 18, 9, 19, 20, 15, 16, 8, 5, 18], [10], [10, 5]]
Y data
[9, 3, 8, 1, 5, 12, 0, 8, 18, 9, 19, 20, 15, 16, 8, 5, 18, 0, 5, 19]


A label (Y data) is simply the next char in a name. That is how all char-based models work at the train phase.

Now we begin adding randomness to the model.
At first, we declare a RAND_Y value, which tells, every which char we replace by a random one. If RAND_Y equals zero, there's no randomness and all labels remain unchained.

In [29]:
def get_Y(dataY, RAND_Y = 0):
  if RAND_Y > 0:
      for i, y in enumerate(dataY):
          if y != 0:
              if np.random.randint(0, RAND_Y) == 0: # on average, we will replace every RAND_Yth char with a randomly selected char
                  dataY[i] = np.random.randint(1, len(chars))
  return dataY

Let's see how the first 10 labels look like after various RAND_Y values.

In [33]:
print("No randomness")
dataY = get_Y(dataY_original, RAND_Y = 0)
RAND_Y = 0
print(f"Original Y: {originalY}")
print(f"Modified Y: {dataY[:20]}")

No randomness
Original Y: [9, 3, 8, 1, 5, 12, 0, 8, 18, 9, 19, 20, 15, 16, 8, 5, 18, 0, 5, 19]
Modified Y: [9, 3, 8, 1, 5, 12, 0, 8, 18, 9, 19, 20, 15, 16, 8, 5, 18, 0, 5, 19]


In [12]:
RAND_Y = 7
print(f"Every {RAND_Y}th modified")
dataY = get_Y(dataY_original, RAND_Y)

print(f"Original Y: {originalY}")
print(f"Modified Y: {dataY[:20]}")

Every 7th modified
Original Y: [9, 3, 8, 1, 5, 12, 0, 8, 18, 9, 19, 20, 15, 16, 8, 5, 18, 0, 5, 19]
Modified Y: [7, 3, 8, 1, 3, 12, 0, 8, 18, 12, 19, 20, 15, 16, 8, 5, 18, 0, 5, 19]


Now we need to prepare data so that they fit an LSTM network.
All steps are basic and self-explanatory.

In [31]:
X = pad_sequences(dataX, maxlen=max_len, dtype='float32')
num_of_chars = max(dataY)
X = X/num_of_chars
Y = np_utils.to_categorical(dataY)
X = X.reshape(X.shape[0], X.shape[1], 1)
print(f"X shape: {X.shape}")
print(f"Y shape: {Y.shape}")

X shape: (115316, 15, 1)
Y shape: (115316, 27)


Let's build our model. Two layers of Bidirectional LSTM with a serious dropout.

In [32]:
def get_model():
  model = Sequential()
  model.add(Bidirectional(LSTM(64, return_sequences=True, input_shape=(X.shape[1], 1))))
  model.add(Dropout(0.5))
  model.add(Bidirectional(LSTM(64)))
  model.add(Dropout(0.5))
  model.add(Dense(Y.shape[1], activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

We've already seen how randomness is added at a training data level. Now let's see how a "simplified Beam" is added at a text generating level.

A simple choice is a greedy serach. After a predict method produces a vector of "probabilities" for each char, we choose the one with the highest value.

In [ ]:
c = model.predict(wordX.reshape(1, max_len, 1), verbose = 0) # a vector
c_max = int(c.argmax(axis=1)) # amvalue with a max probability

An alternative is to choose between a couple of values with the highest score from the probability method. We can randomly choose from a list of all chars using a probability vector as a p parameter in an np.random.choice method.

However, if we have a more or less flat vector of probabilities, like let's say 'a' = 0.05, 'b' = 0.07, 'c' = 0.08 etc. we would end with a chaotic noise rather than any interesting output sounding like a real name.

To avoid this, we can penalize lower probability values and multiply higher values with a simple trick. We take each vector value and raise to power of MULTIPLIER level. Later, we normalize all values so that all probabilities sum up to 1 and can be used as a p parameter in an np.random.choice method.

I usually used MULTIPLIER = 3 value. Let's see how it works.
Let's say we have ten chars in our alphabet. We will generate a vector of probabilities for each char and normalize so that they sum up to 1.



In [ ]:
v = np.random.random(10)
v = v / sum(v)
v = np.sort(v)[::-1]
v = np.round(v, 2)
print(v)
print(np.sum(v[:3]))

Let's assume, that we treat top 2-3 chars as "good", which means that choosing out of them we expect to produce a sensible name that "sounds good". We should somehow minimze the probability of choosing out of the remaining 7-8 chars, but still keep the order of top 2-3 chars in terms of their probability. On the other hand, we do not want to simply cut off the long tail, rather diminish its probability.

Let's see how applying a "simplified Beam" with MULTIPLIER = 3 changes our vector.

In [ ]:
MULTIPLIER = 3

In [ ]:
c = [x ** MULTIPLIER for x in v] #premiujemy największe prawdopodobieństwa
c = [x / sum(c) for x in c]
c = np.round(c, 2)
print(c)
print(np.sum(c[:3]))

As we can see, two top values (0.3 and 0.44) are strengthened and now sum up to 0.93, the other values were made significant.
On the other hand, it's still much more random than greedy search.

So, we are now ready with the model.
We will generate 10 series of names, 10 in each serie.
The model is compiled once at the beginning and it is fit with the data in each series. We will observe how the quality of the generated names gets augmented (or does not?) with each consecitive fit for 10 epochs.



Let's start with a greedy search and no randomnes in training data!

In [39]:
from time import time
names_full = []

not_starting = list(np.arange(26, 33)) # only applies to Polish names
not_starting = [] # empty for English names

model = get_model()

print(f"RandY: {RAND_Y}")
print(f"Greedy search: {RANDOM_CHARS}")
tic = time()

for i in range(SERIES):
    tic = time()
    model.fit(X, Y, epochs=EPOCHS, verbose=0, batch_size=64)
    names_ = []
    if i % 1 == 0:
        for k in range(10):
            n = np.random.randint(1, num_of_chars + 1) #we genearate the first char of the name
            while n in not_starting: #used only for Polish names
                n = np.random.randint(1, num_of_chars + 1)
            n = n/num_of_chars #scaling for an LSTM model

            word = [[]]
            word[0].append(n)
            for j in range(max_len):
                wordX = pad_sequences(word, maxlen=max_len, dtype='float32')
                c = model.predict(wordX.reshape(1, max_len, 1), verbose = 0)
                if RANDOM_CHARS == True:
                    c = list(c.flatten())
                    c = [x ** MULTIPLIER for x in c]
                    c = [x / sum(c) for x in c]
                    c_max = np.random.choice(chars, p = c)
                    c_max = char_to_int[c_max]
                else:
                    c_max = int(c.argmax(axis=1))
                word[0].append(c_max/num_of_chars)
                if c_max == 0:
                    break
            imie = ''.join([int_to_char[int(char*num_of_chars)] for char in word[0]])

            names_.append(imie)

        print(f"Series {i+1:>2} names: {names_} after {(time() - tic)/60:.2f} minutes.")
        names_full.append(names_)

        df = pd.DataFrame(names_full)

print(f'ALL NAMES GENERATED after {time() - tic:.2f} seconds')
df.to_csv('/content/drive/MyDrive/Python_data/English_greedy_norand.csv')



RandY: 0
Greedy search: False
Series  1 names: ['wanesha ', 'tanesha ', 'yanesha ', 'narin ', 'garisha ', 'garisha ', 'parin ', 'quan ', 'sharin ', 'tanesha ']
Series  2 names: ['kamesha ', 'farrin ', 'parin ', 'marin ', 'zanesha ', 'delisha ', 'tanesha ', 'harin ', 'farrin ', 'shanna ']
Series  3 names: ['zachell ', 'harrin ', 'larisha ', 'yachell ', 'larisha ', 'ranina ', 'antell ', 'tarin ', 'nlena ', 'harrin ']
Series  4 names: ['wanesha ', 'istan ', 'marisha ', 'urenta ', 'nrenta ', 'urenta ', 'qunell ', 'yanesha ', 'ranesha ', 'elista ']
Series  5 names: ['elista ', 'jamisha ', 'latan ', 'farris ', 'ista ', 'elista ', 'urenta ', 'nristin ', 'brittin ', 'urenta ']
Series  6 names: ['ranesha ', 'urena ', 'jamina ', 'harrie ', 'yanesha ', 'walina ', 'lataria ', 'ishanna ', 'antnn ', 'charlina ']
Series  7 names: ['isanna ', 'urentin ', 'quanna ', 'gerrica ', 'elisha ', 'gerrica ', 'darricia ', 'latara ', 'urentin ', 'walena ']
Series  8 names: ['yanesha ', 'willian ', 'brittin ', 'h